# Árvores II - Tarefa 2

### 1. Carregar as bases

Vamos carregar as bases lidas na tarefa passada. Se você salvou essas bases em arquivo texto, basta fazer a leitura com o comando ```pd.read_csv``` das seguintes bases:

- X_train
- Y_train
- X_test
- Y_test

Não se esqueça de considerar a leitura dos índices dos arquivos no ```read_csv()```!

In [101]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix

In [102]:
X_train = pd.read_csv('X_train.txt', header=None)
y_train = pd.read_csv('y_train.txt', header=None)
X_test = pd.read_csv('X_test.txt', header=None)
y_test = pd.read_csv('y_test.txt', header=None)
subject_test = pd.read_csv('subject_test.txt', header=None)
subject_train = pd.read_csv('subject_train.txt', header=None)
activity_labels = pd.read_csv('activity_labels.txt', header=None)
features = pd.read_csv('features.txt', header=None, sep=' ')

### 2. Divisão da base em Treino, Validação e Teste

A base já se encontra dividida em Treino e Validação. O que vamos fazer então é extrair uma base de Validação da base de Treino.

Extraia 25% da base de treino como base de validação.

In [103]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=.25, random_state=1729)
X_train.size

5514

In [104]:
X_valid.size

1838

In [105]:
X_test.size

2947

### 3. Melhores 3 variáveis

Rode uma árvore com profundidade máxima igual a 4 para prever a atividade humana com todas as variáveis.
Observe a importância das variáveis e considere as 3 variáveis com maior importância para os próximos passos.
Dica: utilize o atributo ```clf.feature_importances_``` da árvore treinada.

In [106]:
clf = DecisionTreeClassifier(max_depth=4)

In [107]:
X_train.isna().sum()

0    0
dtype: int64

In [108]:
y_train.isna().sum()

0    0
dtype: int64

In [109]:
X_train = pd.get_dummies(X_train, columns=X_train.select_dtypes(include='object').columns)
y_train = pd.get_dummies(X_train, columns=X_train.select_dtypes(include='object').columns)

In [110]:
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=4)

In [111]:
clf.feature_importances_

array([0., 0., 0., ..., 0., 0., 0.])

### 4. Construa uma árvore com as 3 melhores variáveis

Utilizando as três variáveis encontradas acima, construa uma árvore de decisão. Encontre o melhor ```ccp_alpha``` utilizando a base de validação, conforme a estrutura que vimos em aula.

In [112]:
X_valid = pd.get_dummies(X_valid, columns=X_valid.select_dtypes(include='object').columns)
y_valid = pd.get_dummies(y_valid, columns=y_valid.select_dtypes(include='object').columns)

In [113]:
X_train = X_train.iloc[:, :3]
y_train = y_train.iloc[:, :3]
X_valid = X_valid.iloc[:, :3]
y_valid = y_valid.iloc[:, :3]

In [114]:
caminho = DecisionTreeClassifier(random_state=2360873).cost_complexity_pruning_path(X_train,y_train)
ccp_alphas, impurities = caminho.ccp_alphas, caminho.impurities

In [115]:
ccp_alphas

array([0.        , 0.00012088, 0.00012088, 0.00012088])

In [116]:
clfs = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=2360873, ccp_alpha=ccp_alpha).fit(X_train, y_train)
    clfs.append(clf)

In [ ]:
train_scores = [clf.score(X_train, y_train) for clf in clfs]
valid_scores = [clf.score(X_valid, y_valid) for clf in clfs]

fig, ax = plt.subplots()
ax.set_xlabel("alpha")
ax.set_ylabel("Acurácia")
ax.set_title("Acurácia x alpha do conjunto de dados de treino e validação")
ax.plot(ccp_alphas, train_scores, marker='o', label="validação",
        drawstyle="steps-post")
ax.plot(ccp_alphas, valid_scores, marker='o', label="teste",
        drawstyle="steps-post")
ax.legend()
plt.show()